# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import warnings
warnings.filterwarnings("ignore")

# Import API key
#from api_keys import geoapify_key

geoapify_key = "1afe33c004b0421d867634f8cc523ae7"

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(r"C:\Users\nrhl1\OneDrive\Desktop\UTBootcamp\Challenges\python-api-challenge\Starter_Code\output_data\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port elizabeth,-33.9180,25.5701,15.16,88,0,5.66,ZA,2023-07-22 11:33:48
1,1,yellowknife,62.4560,-114.3525,19.29,63,75,3.60,CA,2023-07-22 11:30:32
2,2,hithadhoo,-0.6000,73.0833,28.17,74,71,2.50,MV,2023-07-22 11:30:03
3,3,puerto natales,-51.7236,-72.4875,0.29,76,28,1.67,CL,2023-07-22 11:33:49
4,4,usogorsk,63.4106,48.6872,14.59,88,46,1.32,RU,2023-07-22 11:33:49


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[(city_data_df["Max Temp"] > 10) & (city_data_df["Max Temp"] < 30) & (city_data_df["Wind Speed"]<2) & (city_data_df["Cloudiness"] < 50) & (city_data_df["Humidity"]<50)]

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,conchal,-22.3303,-47.1725,27.08,31,0,1.14,BR,2023-07-22 11:33:52
35,35,laojunmiao,39.8333,97.7333,23.77,17,43,1.23,CN,2023-07-22 11:30:12
87,87,solwezi,-12.1688,26.3894,18.95,40,6,1.92,ZM,2023-07-22 11:34:03
122,122,kapuskasing,49.4169,-82.4331,22.05,43,20,1.54,CA,2023-07-22 11:34:14
223,223,georgievka,49.3314,81.5761,19.89,35,2,1.67,KZ,2023-07-22 11:34:33
243,243,newman,37.3138,-121.0208,29.58,10,42,1.80,US,2023-07-22 11:34:21
250,250,swift current,50.2834,-107.8014,25.45,47,0,1.03,CA,2023-07-22 11:34:37
285,285,lander,42.8330,-108.7307,26.10,34,0,0.00,US,2023-07-22 11:34:43
384,384,mountain home,43.1330,-115.6912,27.46,24,0,0.00,US,2023-07-22 11:34:58
447,447,yinchuan,38.4681,106.2731,27.23,25,2,0.47,CN,2023-07-22 11:31:38


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[['City','Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
23,conchal,BR,-22.3303,-47.1725,31,
35,laojunmiao,CN,39.8333,97.7333,17,
87,solwezi,ZM,-12.1688,26.3894,40,
122,kapuskasing,CA,49.4169,-82.4331,43,
223,georgievka,KZ,49.3314,81.5761,35,
243,newman,US,37.3138,-121.0208,10,
250,swift current,CA,50.2834,-107.8014,47,
285,lander,US,42.8330,-108.7307,34,
384,mountain home,US,43.1330,-115.6912,24,
447,yinchuan,CN,38.4681,106.2731,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
base_url = "https://api.geoapify.com/v2/places"
limit = 1
radius = 10000

params = {
    "limit": limit,  
    "apiKey":geoapify_key,
    "categories": "accommodation.hotel"
}



# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
conchal - nearest hotel: No hotel found
laojunmiao - nearest hotel: No hotel found
solwezi - nearest hotel: Lodge 24
kapuskasing - nearest hotel: Travelodge
georgievka - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
swift current - nearest hotel: Days Inn
lander - nearest hotel: Rodeway Inn Pronghorn Lodge
mountain home - nearest hotel: Best Western Foothills Inn
yinchuan - nearest hotel: Yinchuan hotel
mammoth lakes - nearest hotel: Travelodge by Wyndham Mammoth Lakes
la sarre - nearest hotel: No hotel found
ayaviri - nearest hotel: Hotel San Francisco
kipushi - nearest hotel: No hotel found
jiuquan - nearest hotel: 民族宾馆


,City,Country,Lat,Lng,Humidity,Hotel Name
23,conchal,BR,-22.3303,-47.1725,31,No hotel found
35,laojunmiao,CN,39.8333,97.7333,17,No hotel found
87,solwezi,ZM,-12.1688,26.3894,40,Lodge 24
122,kapuskasing,CA,49.4169,-82.4331,43,Travelodge
223,georgievka,KZ,49.3314,81.5761,35,No hotel found
243,newman,US,37.3138,-121.0208,10,No hotel found
250,swift current,CA,50.2834,-107.8014,47,Days Inn
285,lander,US,42.8330,-108.7307,34,Rodeway Inn Pronghorn Lodge
384,mountain home,US,43.1330,-115.6912,24,Best Western Foothills Inn
447,yinchuan,CN,38.4681,106.2731,25,Yinchuan hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.9,
    color = "City", 
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)